In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as test

In [4]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/rotoncsedu/Transformer/main/spam.csv')
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [8]:
df_spam = df[df['Category'] == 'spam']
df_spam.shape

(747, 2)

In [9]:
df_ham = df[df['Category'] == 'ham']
df_ham.shape

(4825, 2)

In [15]:
df_ham_downsample = df_ham.sample(df_spam.shape[0])
df_ham_downsample.shape

(747, 2)

In [16]:
df_balance = pd.concat([df_ham_downsample, df_spam])
df_balance.shape

(1494, 2)

In [17]:
df_balance['Category'].value_counts()

,count
Category,
ham,747
spam,747


In [18]:
df_balance['spam'] = df_balance['Category'].apply(lambda x: 1 if x == 'spam' else 0)
df_balance.sample(10)

,Category,Message,spam
1672,ham,Glad to see your reply.,0
2189,spam,FREE camera phones with linerental from 4.49/m...,1
476,ham,Ok I'm gonna head up to usf in like fifteen mi...,0
3469,ham,yay! finally lol. i missed our cinema trip las...,0
191,spam,Are you unique enough? Find out from 30th Augu...,1
4642,ham,Havent.,0
3208,ham,This phone has the weirdest auto correct.,0
1688,spam,Free Top ringtone -sub to weekly ringtone-get ...,1
3412,ham,Only once then after ill obey all yours.,0
107,ham,"Aight, I'll hit you up when I get some cash",0


In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balance['Message'], df_balance['spam'], stratify = df_balance['spam'])
X_train.head(5)

,Message
164,-PLS STOP bootydelious (32/F) is inviting you ...
1940,More people are dogging in your area now. Call...
4154,URGENT!! Your 4* Costa Del Sol Holiday or £500...
4603,"Hey j! r u feeling any better, hopeSo hunny. i..."
3126,1st wk FREE! Gr8 tones str8 2 u each wk. Txt N...


In [20]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
